In [ ]:
import pandas as pd   
import funcs
import csv

In [ ]:
# pd.set_option('display.max_columns', None)  # or 1000
# pd.set_option('display.max_rows', None)  # or 1000
#pd.set_option('display.max_colwidth', 170)  # or 199
# pd.reset_option('display.max_columns')
# pd.reset_option('display.max_rows')
#pd.reset_option('display.max_colwidth')
pd.options.display.expand_frame_repr = False

In [ ]:
#Load data from jsonl
jsonObj = pd.read_json(path_or_buf="data/ranking_train.jsonl", lines=True)

#initialization
four_score = []
three_score = []
two_score = []
one_score = []
zero_score = []
df = {"text":[], "comments":[], "lem_comments":[], "score":[]}
text_ar = []
comment_ar = []
score_ar = []
lem_com_ar = []

In [ ]:
#Writing text, comments, scores into arrays and making them in lowercase, lemmatization
#len(jsonObj)
for i in range(0, len(jsonObj)):
    for j in range(0, 5):
        text_ar.append(jsonObj['text'][i].lower())
        comment_ar.append(jsonObj["comments"][i][j]['text'].lower())
        #lem_com_ar.append(funcs.lemmatizer_func(jsonObj["comments"][i][j]['text'].lower()))
        lem_com_ar.append(funcs.text_preprocessing(jsonObj["comments"][i][j]['text'].lower()))
        score_ar.append(jsonObj['comments'][i][j]['score'])

In [ ]:
#Creating dictionary of values for pandas df
data_dict = {"text":text_ar, "comment":comment_ar, "lem_comments":lem_com_ar, "score":score_ar}

#Creating pandas df
df = pd.DataFrame(data_dict)
print(df[['comment', 'lem_comments']])
df.to_csv("data/arr_data.csv", index=False)